In [1]:
from parsing import load_problem, load_solution
from graph_representation import Graph
import time

In [2]:
# weryfikuje poprawność sortowania
def verify_toposort(T, G):
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [81]:
# -*- coding: UTF-8 -*-

from copy import deepcopy
from parsing import load_problem, load_solution
from collections import deque


class Node(object):
    def __init__(self, weight):
        self.weight = weight
        self.successors = []  # lista trójek (numer cyklu, zadanie, operacja)
        self.predecessors = []
        self.next_tech = None
        self.prev_tech = None
        self.next_mach = None
        self.prev_mach = None


class Graph(object):
    def __init__(self, n, m, problem, solution):
        self.n = n
        self.m = m
        self.problem = problem
        self.solution = solution
        self.subgraph = {}
        self.create_subgraph()
        self.graph = deepcopy(self.subgraph)
        self.topological_order = None
        
        for cycle_idx in xrange(m):
            for k, v in self.subgraph.iteritems():
                new_v_name = (cycle_idx + 1,) + k[1:]
                self.graph[new_v_name] = deepcopy(v)
                new_v = self.graph[new_v_name]
                if new_v.next_tech is not None:
                    new_v.next_tech = (cycle_idx + 1,) + new_v.next_tech[1:]
                if new_v.next_mach is not None:
                    new_v.next_mach = (cycle_idx + 1,) + new_v.next_mach[1:]

        for machine in self.solution:
            for cycle_idx in xrange(m):
                self.graph[(cycle_idx,) + machine[-1]].next_mach = ((cycle_idx + 1,) + machine[0])

        for k, v in self.graph.iteritems():            
            if v.next_tech is not None:
                self.graph[v.next_tech].prev_tech = k
            if v.next_mach is not None:
                self.graph[v.next_mach].prev_mach = k

        self.vertices = self.graph.keys()
        self.recalculate_succs_and_preds(self.vertices)
        
    
    def recalculate_succs_and_preds(self, vertices):
        for v_name in vertices:
            v = self.graph[v_name]
            v.successors = {u for u in [v.next_tech, v.next_mach] if u is not None}
            v.predecessors = {u for u in [v.prev_tech, v.prev_mach] if u is not None}
        

    def create_subgraph(self):
        for machine_idx, machine in enumerate(self.solution):
            for pair_idx, (task, operation) in enumerate(machine):
                weights = self.problem[task - 1][operation - 1]  # indeksować od 0?
                        
                if machine_idx + 1 not in weights:
                    raise Exception("Incorrect solution")
                weight = weights[machine_idx + 1]
                
                self.subgraph[(0, task, operation)] = Node(weight)
                v = self.subgraph[(0, task, operation)]
                
                if len(machine) > pair_idx + 1:
                    v.next_mach = ((0,) + machine[pair_idx + 1])
                if len(self.problem[task - 1]) > operation:
                    v.next_tech = ((0, task, operation + 1))
                
                
    # format ruchu taki jak w lower_bound
    # nie sprawdza czy ruch jest poprawny! odpalać tylko na ruchach
    # wygenerowanych przez generate_neighborhood
    def make_a_move(self, (i, m_to, pos)):
        m_from = i[0]
        op = i[1:]
        
        for cycle_idx in xrange(m+1):
            v_name = (cycle_idx,) + i[1:]
            v = self.graph[v_name]
            old_v_next_mach = v.next_mach
            old_v_prev_mach = v.prev_mach
            
            if v.prev_mach is not None:
                self.graph[v.prev_mach].next_mach = v.next_mach
            if v.next_mach is not None:
                self.graph[v.next_mach].prev_mach = v.prev_mach
                
            v.weight = self.problem[op[0]-1][op[1]-1][m_to+1]
            
            _, new_v_next_mach, _, new_v_prev_mach = self.new_succs_preds((i, m_to, pos), cycle=cycle_idx,
                                                                          only_one_cycle=False)
            
            if new_v_next_mach is not None:
                self.graph[new_v_next_mach].prev_mach = v_name
                v.next_mach = new_v_next_mach
            if new_v_prev_mach is not None:
                self.graph[new_v_prev_mach].next_mach = v_name
                v.prev_mach = new_v_prev_mach
                
            to_recalc = [u for u in [v_name, new_v_next_mach, new_v_prev_mach,
                                     old_v_next_mach, old_v_prev_mach] if u is not None]
            self.recalculate_succs_and_preds(to_recalc)
            
        self.solution[m_from].remove(op)
        self.solution[m_to].insert(pos, op)


    def topological_sort(self):
        indegs_cp = {v : len(self.graph[v].predecessors) for v in self.vertices}
        Q = deque([v for v,deg in indegs_cp.items() if deg == 0])
        result = []
        while Q:
            v = Q.pop()
            result.append(v)
            for u in self.graph[v].successors:
                if indegs_cp[u] == 1:
                    Q.appendleft(u)
                indegs_cp[u] -= 1
        
        self.topological_order = result
        
    
    def longest_path_length(self, v1, v2, justH1=False):
        assert self.topological_order is not None
        if v1 not in self.graph or v2 not in self.graph:
            return None
        temp_longest = {v1 : self.graph[v1].weight}

        if v1 == v2:
            return temp_longest[v1]

        for v in self.topological_order[self.topological_order.index(v1)+1:]:
            if (not justH1) or v[0] == 0:
                preds = [p for p in self.graph[v].predecessors if p in temp_longest]
                if preds:
                    best_pred = max(preds, key=lambda p: temp_longest[p] + self.graph[p].weight)
                    temp_longest[v] = temp_longest[best_pred] + self.graph[v].weight
                if v == v2:
                    if v2 in temp_longest:
                        return temp_longest[v2]
                    return None     
                
                
    def new_succs_preds(self, (i,k,s), cycle=0, only_one_cycle=True):
        next_tech = (cycle, i[1], i[2]+1) if i[2] < len(self.problem[i[1]-1]) else None
        prev_tech = (cycle, i[1], i[2]-1) if i[2] > 1 else None
        old_i_machine_idx = self.solution[i[0]].index(i[1:])

        if k != i[0]:
            if s < len(self.solution[k]):
                new_next_m = (cycle,) + self.solution[k][s]
            else:
                if only_one_cycle or cycle == self.m:
                    new_next_m = None
                else:
                    new_next_m = (cycle+1,) + self.solution[k][0]
        else:
            if s < old_i_machine_idx:
                new_next_m = (cycle,) + self.solution[k][s]
            elif s < len(self.solution[k])-1:
                new_next_m = (cycle,) + self.solution[k][s+1]
            else:
                if only_one_cycle or cycle == self.m:
                    new_next_m = None
                else:
                    new_next_m = (cycle+1,) + self.solution[k][0]
                    

        if k != i[0]:
            if s > 0:
                new_prev_m = (cycle,) + self.solution[k][s-1]
            else:
                if only_one_cycle or cycle == 0:
                    new_prev_m = None
                else:
                    new_prev_m = (cycle-1,) + self.solution[k][-1]
        else:
            if s > old_i_machine_idx:
                new_prev_m = (cycle,) + self.solution[k][s]
            elif s > 0:
                new_prev_m = (cycle,) + self.solution[k][s-1]
            else:
                if only_one_cycle or cycle == 0:
                    new_prev_m = None
                else:
                    new_prev_m = (cycle-1,) + self.solution[k][-1]
                
        return next_tech, new_next_m, prev_tech, new_prev_m
    

    def generate_neighborhood(self):
        # ruch jest zły wtw. gdy któraś z tych ścieżek istnieje w starym grafie:
        # następnik technologiczny -> nowy poprzednik maszynowy (path1)
        # nowy następnik maszynowy -> poprzednik technologiczny (path2)
        def valid_move(move):
            next_tech, new_next_m, prev_tech, new_prev_m = self.new_succs_preds(move)

            path1_exists = self.longest_path_length(next_tech, new_prev_m, justH1=True) is not None
            path2_exists = self.longest_path_length(new_next_m, prev_tech, justH1=True) is not None

            return (not path1_exists) and (not path2_exists)
        
        moves = []
        
        for i in [(x[0],) + op for x in enumerate(self.solution) for op in x[1]]:
            for k in [x-1 for x in self.problem[i[1]-1][i[2]-1].keys()]:
                for s in xrange(len(self.solution[k])+1) if k != i[0] else xrange(len(self.solution[k])):
                    move = (i,k,s)
                    if valid_move(move):
                        moves.append(move)
        return moves
    
    
    # m_from, m_to - skąd zabieramy, gdzie wsadzamy
    # op - zabierana operacja, para (nr_maszyny, nr_operacji)
    # pos - pozycja do wsadzenia op na maszynę m_to
    # tutaj numery maszyn idą od 0
    def lower_bound(self, (i, m_to, pos)):
        m_from = i[0]
        op = i[1:]
        
        def alpha(k,i):
            assert i >= 0
            if k != m_from or self.solution[k].index(op) > i:
                return self.solution[k][i]
            #print k, i
            return self.solution[k][i+1]

        # jeśli longest_path nie istnieje, to powinien zwrócić None
        # te drogi poniżej są zawarte w H1 (pierwszej składowej G), więc
        # można by je liczyć trochę szybciej (tylko w H1)

        def R(k,opr):
            return self.longest_path_length((0,) + alpha(k,0), (0,) + opr, justH1=True)

        def Q(k,opr):
            last_op_k_idx = len(self.solution[k]) - 1
            if k == m_from:
                last_op_k_idx -= 1
            return self.longest_path_length((0,) + opr, (0,) + alpha(k, last_op_k_idx), justH1=True)

        def LB(l):
            op_old_weight = self.graph[(0,) + op].weight
            op_new_weight = self.problem[op[0]-1][op[1]-1][m_to+1]

            R1 = lambda: R(l, alpha(m_to, pos-1))
            def R2():
                r = R(l, op)
                return r - op_old_weight if r is not None else None
            Q1 = lambda: Q(l, alpha(m_to, pos))
            def Q2():
                q = Q(l, op)
                return q - op_old_weight if q is not None else None 

            # 0 w maxach po to, żeby mieć jakąś liczbę, gdy drogi nie istnieją
            if pos == 0:
                return max(R2(), 0) + max(Q1(), Q2(), 0) + op_new_weight

            len_m_to = len(self.solution[m_to])
            if m_to == m_from:
                len_m_to -= 1
            if pos == len_m_to:
                return max(R1(), R2(), 0) + max(Q2(), 0) + op_new_weight

            maxR = max(R1(), R2(), 0)
            maxQ = max(Q1(), Q2(), 0)

            return maxR + maxQ + op_new_weight # a może tu ma być op_new_weight?
        
        return max([LB(l) for l in xrange(len(self.solution))])

In [82]:
n, m, problem = load_problem("instances/Barnes_mt10c1.fjs")
solution = load_solution("solutions/Barnes_mt10c1.txt")

G = Graph(n, m, problem, solution)
G.graph[(3,1,4)].successors
G.topological_sort()

In [83]:
G.longest_path_length((0,9,1), (0,1,1))

In [84]:
N = G.generate_neighborhood()

In [85]:
G.make_a_move(((4,10,9), 4, 9))

In [86]:
G.graph[(5,4,4)].predecessors

{(4, 10, 9), (5, 4, 3)}

In [87]:
G.solution

[[(9, 1), (2, 1), (7, 2), (10, 2), (5, 2), (3, 2)],
 [(4, 1),
  (6, 2),
  (7, 1),
  (10, 1),
  (9, 2),
  (5, 3),
  (3, 1),
  (8, 3),
  (1, 2),
  (2, 6)],
 [(6, 1),
  (4, 2),
  (5, 1),
  (8, 1),
  (2, 2),
  (7, 4),
  (10, 3),
  (9, 5),
  (1, 3),
  (3, 4)],
 [(6, 4),
  (7, 3),
  (9, 3),
  (5, 5),
  (3, 3),
  (1, 4),
  (2, 5),
  (10, 8),
  (4, 8),
  (8, 10)],
 [(4, 4),
  (2, 3),
  (5, 6),
  (6, 9),
  (8, 5),
  (1, 5),
  (7, 10),
  (9, 9),
  (3, 10),
  (10, 9)],
 [(6, 3),
  (5, 4),
  (9, 4),
  (7, 6),
  (8, 4),
  (10, 7),
  (1, 6),
  (3, 6),
  (2, 8),
  (4, 10)],
 [(7, 5),
  (4, 5),
  (10, 4),
  (6, 8),
  (9, 7),
  (8, 6),
  (1, 7),
  (2, 7),
  (3, 8),
  (5, 10)],
 [(4, 7),
  (6, 10),
  (5, 8),
  (9, 8),
  (7, 9),
  (3, 7),
  (1, 8),
  (8, 9),
  (2, 9),
  (10, 10)],
 [(6, 5),
  (4, 6),
  (10, 5),
  (5, 7),
  (7, 8),
  (3, 5),
  (8, 7),
  (1, 9),
  (9, 10),
  (2, 10)],
 [(6, 6),
  (7, 7),
  (2, 4),
  (9, 6),
  (10, 6),
  (5, 9),
  (8, 8),
  (4, 9),
  (3, 9),
  (1, 10)],
 [(4, 3), (8, 2), (6

In [88]:
for x in N:
    print x
    print G.lower_bound(x)

((0, 9, 1), 0, 0)
839
((0, 9, 1), 0, 1)
839
((0, 9, 1), 0, 2)
839
((0, 9, 1), 0, 3)
839
((0, 9, 1), 0, 4)
839
((0, 9, 1), 10, 0)
839
((0, 9, 1), 10, 1)
839
((0, 9, 1), 10, 2)
904
((0, 9, 1), 10, 3)
1079
((0, 9, 1), 10, 4)
1108
((0, 2, 1), 0, 0)
882
((0, 2, 1), 0, 1)
763
((0, 2, 1), 0, 2)
763
((0, 2, 1), 0, 3)
763
((0, 2, 1), 0, 4)
763
((0, 2, 1), 0, 5)
950
((0, 2, 1), 10, 0)
773
((0, 2, 1), 10, 1)
763
((0, 2, 1), 10, 2)
802
((0, 2, 1), 10, 3)
1003
((0, 2, 1), 10, 4)
1032
((0, 7, 2), 0, 0)
876
((0, 7, 2), 0, 1)
800
((0, 7, 2), 0, 2)
720
((0, 7, 2), 0, 3)
720
((0, 7, 2), 0, 4)
720
((0, 7, 2), 0, 5)
907
((0, 7, 2), 10, 0)
767
((0, 7, 2), 10, 1)
720
((0, 7, 2), 10, 2)
759
((0, 7, 2), 10, 3)
960
((0, 7, 2), 10, 4)
989
((0, 10, 2), 0, 0)
852
((0, 10, 2), 0, 1)
776
((0, 10, 2), 0, 2)
733
((0, 10, 2), 0, 3)
648
((0, 10, 2), 0, 4)
648
((0, 10, 2), 0, 5)
870
((0, 10, 2), 10, 0)
743
((0, 10, 2), 10, 1)
648
((0, 10, 2), 10, 2)
722
((0, 10, 2), 10, 3)
888
((0, 10, 2), 10, 4)
917
((0, 5, 2), 0, 0)
8